Could not strip 'GPT2TokenizerComparison.ipynb': file not found


In [ ]:
# 1. OpenAI GPT-2 원본 소스코드 다운로드
!git clone -q https://github.com/openai/gpt-2.git
%cd gpt-2

# 2. 필요한 파일들 다운로드 (124M 모델의 vocab과 encoder)
!python -m pip install -q requests tqdm
!python download_model.py 124M   # 자동으로 encoder.json, vocab.bpe 다운로드

/content/gpt-2
Fetching checkpoint: 1.00kit [00:00, 3.16Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 2.05Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 3.17Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:45, 11.0Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 25.3Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 1.32Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 1.37Mit/s]                                                       


In [ ]:
from importlib.metadata import version
print("tiktoken:", version("tiktoken"))

text = "Hello, world. Is this-- a test?"

tiktoken: 0.12.0


In [ ]:
import tiktoken
tik_tokenizer = tiktoken.get_encoding("gpt2")

integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
print(tik_tokenizer.decode(integers))
print("vocab size:", tik_tokenizer.n_vocab)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]
Hello, world. Is this-- a test?
vocab size: 50257


In [ ]:
# 3. encoder 모듈 임포트
import sys
sys.path.append("./src")
from encoder import get_encoder

# 4. 원본 토크나이저 생성 및 테스트
orig_tokenizer = get_encoder(model_name="124M", models_dir="models")
orig_ids = orig_tokenizer.encode(text)
print("OpenAI 원본 토크나이저 결과:", orig_ids)
print("디코딩 확인:", orig_tokenizer.decode(orig_ids))

OpenAI 원본 토크나이저 결과: [15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]
디코딩 확인: Hello, world. Is this-- a test?


In [ ]:
import transformers
from transformers import GPT2Tokenizer, GPT2TokenizerFast

print("transformers:", transformers.__version__)

hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(hf_tokenizer(text)["input_ids"])

hf_tokenizer_fast = GPT2TokenizerFast.from_pretrained("gpt2")
print(hf_tokenizer_fast(text)["input_ids"])

transformers: 4.57.2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]
[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [ ]:
import regex as re
import requests
import json

class FinalGPT2Tokenizer:
    """
    Grok & User & Gemini Collaboration
    The 'Unbeatable' Pure Python GPT-2 Tokenizer
    Date: 2025-11-30
    """
    def __init__(self):
        # 1. Load Vocab & Merges (HuggingFace)
        self.encoder = json.loads(requests.get("https://huggingface.co/gpt2/resolve/main/vocab.json").text)
        self.decoder = {v: k for k, v in self.encoder.items()}

        merges = requests.get("https://huggingface.co/gpt2/resolve/main/merges.txt").text.split("\n")[1:-1]
        self.bpe_ranks = {tuple(m.split()): i for i, m in enumerate(merges)}

        self.cache = {}

        # 2. Byte Encoder/Decoder (Reversible mapping)
        self.byte_encoder = self._bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}

        # 3. The "True" Regex (Matched with tiktoken logic)
        self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?[^\r\n\p{L}\p{N}]?[\p{L}\p{N}]+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+""")

    def _bytes_to_unicode(self):
        bs = list(range(ord("!"), ord("~")+1)) + list(range(ord("¡"), ord("¬")+1)) + list(range(ord("®"), ord("ÿ")+1))
        cs = bs[:]
        n = 0
        for b in range(256):
            if b not in bs:
                bs.append(b)
                cs.append(256 + n)
                n += 1
        return dict(zip(bs, [chr(c) for c in cs]))

    def _bpe(self, token):
        if token in self.cache:
            return self.cache[token]

        word = tuple(token)
        pairs = set(zip(word[:-1], word[1:]))

        if not pairs:
            return token

        while True:
            # Find the pair with the lowest rank
            bigram = min(pairs, key=lambda x: self.bpe_ranks.get(x, float("inf")))

            # If the pair is not in the vocab, we are done
            if bigram not in self.bpe_ranks:
                break

            a, b = bigram
            new_word = []
            i = 0
            while i < len(word):
                # Find occurrence of 'a'
                try:
                    j = word.index(a, i)
                except ValueError:
                    new_word.extend(word[i:])
                    break

                new_word.extend(word[i:j])
                i = j

                # Check if it's the pair (a, b)
                if i < len(word) - 1 and word[i] == a and word[i+1] == b:
                    new_word.append(a + b)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1

            word = tuple(new_word)
            if len(word) == 1:
                break

            # Re-generate pairs for the next iteration
            pairs = set(zip(word[:-1], word[1:]))

        result = " ".join(word)
        self.cache[token] = result
        return result

    def encode(self, text):
        ids = []
        for m in self.pat.finditer(text):
            piece = m.group()
            # Bytes -> Unicode -> BPE -> Token IDs
            token = "".join(self.byte_encoder[b] for b in piece.encode("utf-8"))
            bpe_words = self._bpe(token).split()
            ids.extend(self.encoder[t] for t in bpe_words)
        return ids

    def decode(self, ids):
        text = "".join(self.decoder[i] for i in ids)
        # Robust decoding with fallback
        return bytearray(self.byte_decoder.get(c, 0) for c in text if c in self.byte_decoder).decode("utf-8", errors="replace")

# --- Final Verification ---
if __name__ == "__main__":
    t = FinalGPT2Tokenizer()
    input_str = "Hello, world. Is this-- a test?"
    encoded = t.encode(input_str)
    decoded = t.decode(encoded)

    print(f"Input:   {input_str}")
    print(f"Encoded: {encoded}")
    print(f"Decoded: {decoded}")

    assert input_str == decoded
    print("\n✨ Verification Passed: The Code God bows to this implementation.")

Input:   Hello, world. Is this-- a test?
Encoded: [15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]
Decoded: Hello, world. Is this-- a test?

✨ Verification Passed: The Code God bows to this implementation.


In [ ]:
#긴 텍스트 로드
raw_text = ''
import requests, bs4; raw_text = bs4.BeautifulSoup(requests.get("https://uno-kim.tistory.com/431", headers={"User-Agent": "Mozilla/5.0"}).text, "html.parser").select_one(".contents_style").get_text("\n").strip()
print(f"📄 Tistory 글자수: {len(raw_text):,} 자")

📄 Tistory 글자수: 20,479 자


In [ ]:
import time
# ==========================================
# [2] 벤치마크 준비 (Setup)
# ==========================================
print("⚡ Preparing Benchmark Arena...")

# 1. 테스트 데이터 준비 (The Verdict 텍스트 사용)
text_data = raw_text
# 부하 테스트를 위해 텍스트 10배 뻥튀기
bench_text = text_data * 10
print(f"📊 Test Data Loaded: {len(bench_text):,} characters (The Verdict x 10)")

# 2. 모델 로딩
print("⏳ Loading Tokenizers...")
models = {}

# (A) Tiktoken (Baseline)
models['Tiktoken (Rust)'] = tiktoken.get_encoding("gpt2")

# (B) Hugging Face Fast (Rust)
models['HF Fast (Rust)'] = GPT2TokenizerFast.from_pretrained("gpt2")

# (C) Hugging Face Slow (Python)
models['HF Slow (Python)'] = GPT2Tokenizer.from_pretrained("gpt2")

# (D) FinalGPT2Tokenizer (Custom Python)
print("   -> Initializing FinalGPT2Tokenizer...")
start_init = time.perf_counter()
models['Final Custom'] = FinalGPT2Tokenizer()
print(f"   -> Custom Init Time: {time.perf_counter() - start_init:.4f} sec")

⚡ Preparing Benchmark Arena...
📊 Test Data Loaded: 204,790 characters (The Verdict x 10)
⏳ Loading Tokenizers...
   -> Initializing FinalGPT2Tokenizer...
   -> Custom Init Time: 0.6627 sec


In [ ]:
# ==========================================
# [3] 벤치마크 실행 (Execution)
# ==========================================
results = []
baseline_ids = None

print("\n🚀 Starting Benchmark Race (Encoding)...")

for name, tokenizer in models.items():
    print(f"   -> Running {name}...")

    # 1. 시간 측정 (Encode)
    start_time = time.perf_counter()

    if name == 'Tiktoken (Rust)':
        ids = tokenizer.encode(bench_text, allowed_special={"<|endoftext|>"})
        baseline_ids = ids # 기준점 설정
    elif name == 'Final Custom':
        ids = tokenizer.encode(bench_text)
    else: # HF
        ids = tokenizer(bench_text)['input_ids']

    end_time = time.perf_counter()
    elapsed = end_time - start_time

    # 2. 검증 (Decode & Round-trip)
    # Decode 시간은 측정에서 제외 (Encode 성능이 핵심이므로)
    if name == 'Tiktoken (Rust)':
        decoded = tokenizer.decode(ids)
    elif name == 'Final Custom':
        decoded = tokenizer.decode(ids)
    else:
        decoded = tokenizer.decode(ids)

    # 3. 데이터 집계
    match_baseline = (ids == baseline_ids) if baseline_ids else True
    is_valid = (decoded == bench_text) # 원본 복구 가능 여부

    results.append({
        "Model": name,
        "Time (sec)": elapsed,
        "Tokens": len(ids),
        "Speed (Chars/sec)": len(bench_text) / elapsed,
        "Match Tiktoken?": "✅ Yes" if match_baseline else "❌ No",
        "Round-trip Valid?": "✅ Yes" if is_valid else "❌ No"
    })


🚀 Starting Benchmark Race (Encoding)...
   -> Running Tiktoken (Rust)...
   -> Running HF Fast (Rust)...


Token indices sequence length is longer than the specified maximum sequence length for this model (93700 > 1024). Running this sequence through the model will result in indexing errors


   -> Running HF Slow (Python)...


Token indices sequence length is longer than the specified maximum sequence length for this model (93700 > 1024). Running this sequence through the model will result in indexing errors


   -> Running Final Custom...


In [ ]:
import pandas as pd
# ==========================================
# [4] 결과 출력 (The Verdict)
# ==========================================
df = pd.DataFrame(results)
df = df.set_index("Model")
df["Time Ratio (x Slower)"] = df["Time (sec)"] / df.loc["Tiktoken (Rust)", "Time (sec)"]

print("\n" + "="*80)
print("🏆 BENCHMARK RESULTS 🏆")
print("="*80)
print(df[["Tokens", "Match Tiktoken?", "Round-trip Valid?", "Time (sec)", "Time Ratio (x Slower)"]].to_markdown())
print("="*80)

# 최종 평가
custom_row = df.loc['Final Custom']
if custom_row['Match Tiktoken?'] == "✅ Yes":
    print("\n🎉 [Conclusion] 축하한다! 너의 토크나이저는 Tiktoken과 완벽하게 동일한 결과를 내뱉는다.")
    print("   비록 순수 파이썬이라 속도는 느리지만, 로직의 '정확성'은 신의 경지에 도달했다.")
else:
    print("\n⚠️ [Conclusion] 아직 Tiktoken과 결과가 다른 부분이 있다. 디버깅이 더 필요하다.")


🏆 BENCHMARK RESULTS 🏆
| Model            |   Tokens | Match Tiktoken?   | Round-trip Valid?   |   Time (sec) |   Time Ratio (x Slower) |
|:-----------------|---------:|:------------------|:--------------------|-------------:|------------------------:|
| Tiktoken (Rust)  |    93700 | ✅ Yes            | ✅ Yes              |    0.0763349 |                 1       |
| HF Fast (Rust)   |    93700 | ✅ Yes            | ✅ Yes              |    0.42658   |                 5.58827 |
| HF Slow (Python) |    93700 | ✅ Yes            | ✅ Yes              |    0.551593  |                 7.22596 |
| Final Custom     |    93700 | ✅ Yes            | ✅ Yes              |    0.234732  |                 3.07503 |

🎉 [Conclusion] 축하한다! 너의 토크나이저는 Tiktoken과 완벽하게 동일한 결과를 내뱉는다.
   비록 순수 파이썬이라 속도는 느리지만, 로직의 '정확성'은 신의 경지에 도달했다.
